In [1]:
from imports import *

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [11]:
(0.1/2)**(2*0+1)

0.05

In [2]:
from math import factorial as fact

def modified_bessel_term(m,v,u):
    return (1/(fact(m)*fact(m+v)))*((u/2)**(2*m+v))

def modified_bessel_first_kind_approx(v,u,n_terms=100):
    assert isinstance(v, int)
    tot=0
    for m in range(0,n_terms):
        tot += modified_bessel_term(m,v,u)
    return tot
modified_bessel_first_kind_approx(1,0.01,n_terms=2000)

res = []
for v in tqdm(range(1,31)):
    for u in [0.0000000000000001,0.0000001,0.00001,0.0001,0.001,0.01,0.1]:
        for n_terms in [1,2,3,4,5,6,10,100,1000]:
            mod = modified_bessel_first_kind_approx(v,u,n_terms)
            res += [[v,u,n_terms,mod]]

100%|██████████| 30/30 [00:14<00:00,  1.96it/s]


In [3]:
df = pd.DataFrame(res, columns=['v','u','n_terms','out'])
df

,v,u,n_terms,out
0,1,1.000000e-16,1,5.000000e-17
1,1,1.000000e-16,2,5.000000e-17
2,1,1.000000e-16,3,5.000000e-17
3,1,1.000000e-16,4,5.000000e-17
4,1,1.000000e-16,5,5.000000e-17
5,1,1.000000e-16,6,5.000000e-17
6,1,1.000000e-16,10,5.000000e-17
7,1,1.000000e-16,100,5.000000e-17
8,1,1.000000e-16,1000,5.000000e-17
9,1,1.000000e-07,1,5.000000e-08


In [ ]:
class Laplace():
    def __init__(self):
        self.mus = None
        self.bs = None
        self.ws = None
        self.sigmas = None
        self.mus = None
        self.cuda = None
        self.initialized = False
        
    @property
    def shape(self):
        assert self.initialized == True
        print('ws shape: ', self.ws.shape)
        print('mus shape: ', self.mus.shape)
        print('sigmas shape: ', self.sigmas.shape)
        
    def load(self, ws, sigmas, mus):
        self.d = mus.shape[3]
        self.n_gaussians = ws.shape[1]
        self.ws = ws
        self.sigmas = sigmas
        self.mus = mus
        self.initialized = True
        print('GMM Loaded and initialized')

    def initialize(self, d, n_gaussians, sigma, mu_min, mu_max, cuda, initialization = 'random'):
        assert initialization in ['random','orthogonal']
        
        self.d = d
        self.n_gaussians = n_gaussians
        self.initialization = initialization
        self.cuda = cuda
        if initialization == 'random':
            self.ws, self.mus, self.sigmas = make_random_GMM(d, n_gaussians, sigma, mu_min, mu_max, cuda)
        elif initialization == 'orthogonal':
            self.ws, self.mus, self.sigmas = make_orthogonal_GMM(d, n_gaussians, sigma, cuda)
        print('GMM Initialized')
        self.initialized = True
    
    def sample(self,n_points = 1):
        assert self.initialized == True
        print(self.mus.shape)
        mus = self.mus.repeat(1,1,n_points,1)
        sigmas = self.sigmas.repeat(1,1,n_points,1)
        chosen_gaussians = torch.multinomial(self.ws, n_points, replacement=True)

        batch_size = self.ws.shape[0]
        chosen_gaussians = chosen_gaussians.view(batch_size, 1, n_points,1).repeat(1,1,1,self.d)

        ep = torch.randn(batch_size, 1, n_points,self.d)
        if self.cuda:
            ep = ep.cuda()
        print(mus.shape, chosen_gaussians.shape)
        u_mu = torch.gather(mus, 1, chosen_gaussians)
        u_sigma = torch.gather(sigmas, 1, chosen_gaussians)
        u_i = u_mu + ep*u_sigma
        return u_i
        
    def tsne_plot(self, n_points=200):
        assert self.initialized == True
        plt.figure(figsize=(10,10))
        plt.title('Latent space - TSNE')
        plt.xlabel('Dimension 1')
        plt.ylabel('Dimension 2')
        samples = self.sample(n_points)
        samples = sample_from_gmm(n_points, self.ws, self.mus, self.sigmas, self.cuda)
        samples = samples.squeeze(0).squeeze(0)
        
        tsne = TSNE(n_components=2, perplexity=20)
        gmm_tsne = tsne.fit_transform(samples.cpu().detach().numpy())
        plt.scatter(*gmm_tsne.T, c='blue', marker='o', alpha=0.5)
        
    def pdf(self, u):
        #get the probability density function(s) at point(s) u
        #for a mixed gaussian model with weights given by self.ws,
        #means given by self.mus and diagonal covariances given by self.sigmas**2

        #this is the exponent of the multivariate gaussian
        #The equation for a multivariate gaussian is :
        #  (2*pi)^(-d/2)*det(cov)^(-1/2)*e^((1/2)*(x-mu).T*inverse_cov*(x-mu))
        #in our case the calculation is simplified because
        #we assume the covariance matrix is diagonal, so the determinant is just the product
        #of the elements, and the inverse is just 1/sigma
        ws = self.ws.unsqueeze(2).unsqueeze(3)
        covs = self.sigmas**2
        
        #to avoid infinities - use log covariance
        log_covs = torch.log(covs)
        
        log_det_cov = (torch.sum(log_covs, 2)).unsqueeze(2)
        exponent = (-1/2)*((u-self.mus)*(1/covs)*(u-self.mus)).sum(dim=2, keepdim=True)
                
        if self.cuda:
            ws = ws.type(torch.cuda.DoubleTensor)
            log_det_cov = log_det_cov.type(torch.cuda.DoubleTensor)
            exponent = exponent.type(torch.cuda.DoubleTensor)
        else:
            ws = ws.type(torch.DoubleTensor)
            log_det_cov = log_det_cov.type(torch.DoubleTensor)
            exponent = exponent.type(torch.DoubleTensor)
        
        #multiplier = ((2*np.pi)**(-self.d/2))*np.e**((-1/2)*log_det_cov)
        multiplier = np.e**((-1/2)*log_det_cov)#((det_cov)**(-1/2))
        
        if self.cuda:
            multiplier = multiplier.type(torch.cuda.DoubleTensor)
        else:
            multiplier = multiplier.type(torch.DoubleTensor)
        pdf = multiplier*np.e**exponent
        #since ours is a mixture of gaussians, we sum the pdf over the number of gaussians we have 
        #multiplied by the weight each gaussian has
        pdf = ((ws*pdf).sum(dim=1,keepdim=True))#.type(torch.cuda.DoubleTensor)   
        return pdf


    
d = 30
n_gaussians = 6
sigma = 1
mu_min = -50
mu_max = 50
cuda = True
n_points=1000
        
q = GMM()
q.initialize(d, n_gaussians, sigma, mu_min, mu_max, cuda, initialization='orthogonal')
q.tsne_plot(n_points=500)
k = q.sample(n_points=500)
q.pdf(k)
#p = GMM()
#p.initialize(d, n_gaussians, sigma, mu_min, mu_max, cuda, initialization='random')
#p.tsne_plot(n_points=500)